In [2]:
import psycopg2
import pandas as pd
import re
from IPython.display import clear_output
from transformers import AutoTokenizer, AutoModel
import matplotlib.pyplot as plt
from math import ceil


# Função para se conectar ao banco
def get_connection():
    return psycopg2.connect(
        dbname="PROCESSOS",
        user="",
        password="",
        host="",
        port="5432"
    )


## 1 - Limpando o número do processo e criando um campo novo, que contem o numero do processo e o tribunal, e permitirá a identificação unívoca do processo

In [3]:

# Função para limpar o número do processo
def limpar_numero(numero):
    if numero:
        return re.sub(r'\D', '', numero)
    return ''

# Função para buscar, processar e atualizar os dados com feedback de progresso
def atualizar_processos():
    conn = get_connection()
    cursor = conn.cursor()

    # Carrega os dados
    query = "SELECT numero_processo, tribunal FROM processos_jusbrasil_chave_dia where numero_processo_tribunal is null"
    df = pd.read_sql_query(query, conn)

    # Limpa e cria os novos campos
    df['numero_processo_limpo'] = df['numero_processo'].apply(limpar_numero)
    df['numero_processo_tribunal'] = df['numero_processo_limpo'] + '_' + df['tribunal']

    total = len(df)

    # Atualiza linha por linha com contador
    for i, (_, row) in enumerate(df.iterrows(), start=1):
        print(f"{i}/{total}")

        cursor.execute("""
            UPDATE processos_jusbrasil_chave_dia
            SET numero_processo_limpo = %s,
                numero_processo_tribunal = %s
            WHERE numero_processo = %s AND tribunal = %s
        """, (
            row['numero_processo_limpo'],
            row['numero_processo_tribunal'],
            row['numero_processo'],
            row['tribunal']
        ))
        conn.commit()
        clear_output(wait=True)
        
    cursor.close()
    conn.close()
    print("Atualização concluída com sucesso.")

In [ ]:
atualizar_processos()

3294/24166


# Copiando os processos únicos para a tabela que será utilizada

In [ ]:
def criar_tabela_unicos():
    conn = get_connection()
    cursor = conn.cursor()

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS processos (
            id SERIAL PRIMARY KEY,
            tribunal TEXT,
            apelacao TEXT,
            resumo_ementa TEXT,
            titulo TEXT,
            ementa_completa TEXT,
            numero_processo TEXT,
            apelante TEXT,
            reu TEXT,
            numero_processo_limpo TEXT,
            numero_processo_tribunal TEXT UNIQUE,
            start_date DATE,
            end_date DATE
        )
    """)

    conn.commit()
    cursor.close()
    conn.close()

def inserir_processos_unicos():
    conn = get_connection()
    cursor = conn.cursor()

    query = """
        SELECT DISTINCT ON (numero_processo_tribunal)
            tribunal, apelacao, resumo_ementa, titulo, ementa_completa,
            numero_processo, apelante, reu,
            numero_processo_limpo, numero_processo_tribunal,
            start_date, end_date
        FROM public.processos_jusbrasil_chave_dia
        WHERE numero_processo_tribunal IS NOT NULL
        ORDER BY numero_processo_tribunal, LENGTH(ementa_completa) DESC;

    """

    df = pd.read_sql_query(query, conn)

    for _, row in df.iterrows():
        try:
            cursor.execute("""
                INSERT INTO processos (
                    tribunal, apelacao, resumo_ementa, titulo, ementa_completa,
                    numero_processo, apelante, reu,
                    numero_processo_limpo, numero_processo_tribunal,
                    start_date, end_date
                ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                ON CONFLICT (numero_processo_tribunal) DO NOTHING
            """, (
                row['tribunal'], row['apelacao'], row['resumo_ementa'], row['titulo'], row['ementa_completa'],
                row['numero_processo'], row['apelante'], row['reu'],
                row['numero_processo_limpo'], row['numero_processo_tribunal'],
                row['start_date'], row['end_date']
            ))
        except Exception as e:
            print(f"Erro ao inserir: {e}")

    conn.commit()
    cursor.close()
    conn.close()
    print("Inserção concluída.")




In [ ]:
# Executa
criar_tabela_unicos()
inserir_processos_unicos()

## Contando a qtd de tokens de cada ementa_completa e armazenando no banco (este campo será útil para posteriormente avaliar o tamanho das ementas

In [ ]:
# Tokenizador e modelo carregados localmente
tokenizer = AutoTokenizer.from_pretrained("C:/Users/Loreane/Documents/bert")
model = AutoModel.from_pretrained("C:/Users/Loreane/Documents/bert")


# Atualiza o campo qtd_tokens_ementa
def atualizar_qtd_tokens():
    conn = get_connection()
    cursor = conn.cursor()

    # Seleciona todos os registros
    df = pd.read_sql_query("SELECT numero_processo_tribunal, ementa_completa FROM processos where qtd_tokens_ementa is null", conn)

    for i, row in df.iterrows():
        #atribuindo o conteúdo da ementa na variavel texto, e vazio caso ela seja nula
        texto = row["ementa_completa"] or ""
        #Remover os espaços em branco do início e final e sequencias em branco
        texto_limpo = re.sub(r'\s+', ' ', texto.strip())
        qtd_tokens = len(tokenizer.tokenize(texto_limpo))
        clear_output(wait=True)
        cursor.execute("""
            UPDATE processos
            SET qtd_tokens_ementa = %s
            WHERE numero_processo_tribunal = %s
        """, (qtd_tokens, row["numero_processo_tribunal"]))

        print(f"{i+1}/{len(df)} - Processo {row['numero_processo_tribunal']}: {qtd_tokens} tokens")

    conn.commit()
    cursor.close()
    conn.close()
    print("Atualização concluída com sucesso.")


atualizar_qtd_tokens()
